# Imports

In [10]:
# General Imports
import pandas as pd
import matplotlib.pyplot as plt
import random
import geopandas
import copy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset
from torchvision.transforms import transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor
from torch.autograd import Variable

import warnings
import random
import time

import skimage
from skimage.transform import rescale
import cv2

from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error

import wandb

In [11]:
# local imports
from dataloader_SRCNN import Dataset

# Define Paths

In [3]:
# define paths
spot6_mosaic = '/home/simon/CDE_UBS/thesis/data_collection/spot6/spot6_mosaic.tif'
spot6_path = "/home/simon/CDE_UBS/thesis/data_collection/spot6/"
sen2_path = "/home/simon/CDE_UBS/thesis/data_collection/sen2/merged_reprojected/"
closest_dates_filepath = "/home/simon/CDE_UBS/thesis/data_loader/data/closest_dates.pkl"

# get dataset object
dataset = Dataset(spot6_mosaic,sen2_path,spot6_path,closest_dates_filepath,window_size=500,factor=(10/1.5))
loader = DataLoader(dataset,batch_size=1, shuffle=True, num_workers=1)
print("Loader Length: ",len(loader))

precalculated dataset file found, loading...

Dataset Object successfully instanciated!
Loader Length:  68150


# Define Model

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [6]:
class SRCNN(nn.Module):
    # https://keras.io/examples/vision/super_resolution_sub_pixel/
    # https://mfarahmand.medium.com/cnn-based-single-image-super-resolution-6ffcd39ec993
    def __init__(self, num_channels=3):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=9, padding=9 // 2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=5 // 2)
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size=5, padding=5 // 2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x
    
"""  
def loss_func(a, b):
    # PSNR
    # https://www.geeksforgeeks.org/python-peak-signal-to-noise-ratio-psnr/#:~:text=Peak%20signal%2Dto%2Dnoise%20ratio%20(PSNR)%20is%20the,with%20the%20maximum%20possible%20power.
    # PSNR = 20*log(max(max(f)))/((MSE)^0.5)
    # result in db
    # lower->better
    try:
        import math
        mse = np.mean((a - b) ** 2)
        if(mse == 0):  # MSE is zero means no noise is present in the signal. Therefore PSNR have no importance.
            return 100
        max_pixel = 255.0
        psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
        return(psnr)
    except ValueError:
        return(0)
        
"""

"""
def loss_func(a,b):
    # MAE
    error = torch.abs(a - b).sum().data
    error = Variable(error.data, requires_grad=True)
    return(error)
"""

def loss_func(a,b):
    # MSE
    loss = nn.MSELoss()
    mse = loss(a, b)
    mse = Variable(mse.data, requires_grad=True)
    return(mse)
    
model = SRCNN()

In [7]:
# implementation of model trainer function
def train_model(model,batch_size=1,lr=0.01,epochs=10,wandb_name="test"):
    
    logging=True
    if logging==True:
        wandb.init(project=wandb_name, entity="simon-donike")
        wandb.config = {
          "learning_rate": lr,
          "epochs": epochs,
          "batch_size": batch_size
        }
    
    # define loaders
    loader_train = DataLoader(dataset,batch_size=batch_size, shuffle=True, num_workers=1)
    loader_test  = DataLoader(dataset,batch_size=batch_size, shuffle=True, num_workers=1)
    loader_full  = DataLoader(dataset,batch_size=batch_size, shuffle=True, num_workers=1)


    train_loss = []  # where we keep track of the training loss
    train_accuracy = []  # where we keep track of the training accuracy of the model
    val_loss = []  # where we keep track of the validation loss
    val_accuracy = []  # where we keep track of the validation accuracy of the model
    epochs = epochs  # number of epochs

    # initialize model

    model = model.double()
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    for e in range(epochs):
        model.train()
        train_correct = 0
        for (x_train_batch, y_train_batch) in loader_train:
            x_train_batch = x_train_batch.to(torch.double)
            x_train_batch = x_train_batch.to(device)

            y_train_batch = y_train_batch.to(torch.double) 
            y_train_batch = y_train_batch.to(device)
            y_hat = model(x_train_batch)  # forward pass

            loss = loss_func(y_hat, y_train_batch)  # compute the loss

            loss.backward()  # obtain the gradients with respect to the loss
            optimizer.step()  # perform one step of gradient descent
            optimizer.zero_grad()  # reset the gradients to 0
            y_hat_class = torch.argmax(y_hat.detach(), axis=1)  # we assign an appropriate label based on the network's prediction
            train_correct += torch.sum(y_hat_class == y_train_batch)
            train_loss.append(loss.item() / len(x_train_batch))
            if logging==True:
                wandb.log({'loss': loss.item() / len(x_train_batch)})

        train_accuracy.append(train_correct / len(loader_train.dataset))
        if logging==True:
            wandb.log({'train_acc': train_correct / len(loader_train.dataset)})


        print ('Epoch', e+1, ' finished.')

    if logging==True:
        wandb.finish()

In [9]:
train_model(model=model,batch_size=1,lr=0.01,epochs=25,wandb_name="SRCNN")

loss,▃▂▂▂▁▃▃▁▂▃▂▁▁▂▂▂▃▄▂▂▃▃▂▂█▃▂▁▂▂▂▂▁▂▃▃▃▂▄▂
loss,32710.13632


wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Problem at: <ipython-input-7-4b7161b777ac> 6 train_model


KeyboardInterrupt: 

In [27]:
loader = DataLoader(dataset,batch_size=2, shuffle=True, num_workers=0)

In [28]:
%%timeit
for i in loader:
    a,b = i
    break

912 ms ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
a,b = dataset.__getitem__(random.randint(0,1000))

331 ms ± 106 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
